In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

In [ ]:
!ls /content/gdrive/MyDrive/Dataset/MRNet-v1.0  # Replace with your actual directory path

In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
# Define the function to load data
def load_data(folder_path):
 data = []
 for filename in os.listdir(folder_path):
  if filename.endswith(".npy"):
    file_path = os.path.join(folder_path, filename)
    loaded_data = np.load(file_path, allow_pickle=True)
    data.append(loaded_data[0])
 data = np.array(data)
 return data

In [ ]:
axial_data = load_data("/content/gdrive/MyDrive/Dataset/MRNet-v1.0/train/axial")
coronal_data = load_data("/content/gdrive/MyDrive/Dataset/MRNet-v1.0/train/coronal")
sagittal_data = load_data("/content/gdrive/MyDrive/Dataset/MRNet-v1.0/train/sagittal")
print(axial_data.shape)


In [ ]:
# Flatten the data
axial_flat = axial_data.reshape(axial_data.shape[0], axial_data.shape[1], axial_data.shape[2], 1)
coronal_flat = coronal_data.reshape(coronal_data.shape[0], coronal_data.shape[1], coronal_data.shape[2], 1)
sagittal_flat = sagittal_data.reshape(sagittal_data.shape[0], sagittal_data.shape[1], sagittal_data.shape[2], 1)

In [ ]:
# Load labels
acl_annotation = pd.read_csv("/content/gdrive/MyDrive/Dataset/MRNet-v1.0/train-acl.csv", header=None)
acl_label = acl_annotation.iloc[:, 1].values
print(acl_label.shape)
abnormal_annotation = pd.read_csv("/content/gdrive/MyDrive/Dataset/MRNet-v1.0/train-abnormal.csv", header=None)
abnormal_label = abnormal_annotation.iloc[:, 1].values
meniscus_annotation = pd.read_csv("/content/gdrive/MyDrive/Dataset/MRNet-v1.0/train-meniscus.csv", header=None)
meniscus_label = meniscus_annotation.iloc[:, 1].values

In [ ]:
axial_test_data = load_data("/content/gdrive/MyDrive/Dataset/MRNet-v1.0/valid/axial")
coronal_test_data = load_data("/content/gdrive/MyDrive/Dataset/MRNet-v1.0/valid/coronal")
sagittal_test_data = load_data("/content/gdrive/MyDrive/Dataset/MRNet-v1.0/valid/sagittal")


In [ ]:
#Flatten the test data
axial_test_flat = axial_test_data.reshape(axial_test_data.shape[0], axial_test_data.shape[1], axial_test_data.shape[2], -1)
coronal_test_flat = coronal_test_data.reshape(coronal_test_data.shape[0], coronal_test_data.shape[1], coronal_test_data.shape[2], -1)
sagittal_test_flat = sagittal_test_data.reshape(sagittal_test_data.shape[0], sagittal_test_data.shape[1], sagittal_test_data.shape[2], -1)

In [ ]:
# Load test labels
acl_test_annotation = pd.read_csv("/content/gdrive/MyDrive/Dataset/MRNet-v1.0/valid-acl.csv",header=None)
acl_test_label = acl_test_annotation.iloc[:, 1].values
abnormal_test_annotation = pd.read_csv("/content/gdrive/MyDrive/Dataset/MRNet-v1.0/valid-abnormal.csv",header=None)
abnormal_test_label = abnormal_test_annotation.iloc[:, 1].values
meniscus_test_annotation = pd.read_csv("/content/gdrive/MyDrive/Dataset/MRNet-v1.0/valid-meniscus.csv",header=None)
meniscus_test_label = meniscus_test_annotation.iloc[:, 1].values

In [ ]:
# Combine ABNORMAL data from different planes
X_train_ABNORMAL = np.concatenate((axial_flat, coronal_flat, sagittal_flat))
X_test_ABNORMAL = np.concatenate((axial_test_flat, coronal_test_flat, sagittal_test_flat))


In [ ]:
# Define the CNN model
model = tf.keras.Sequential([
 tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Flatten(),
 tf.keras.layers.Dense(512, activation='relu'),
 tf.keras.layers.Dropout(0.5),
 tf.keras.layers.Dense(1, activation='sigmoid')
])


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Train the model
model.fit(X_train_abnormal, abnormal_label, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test_ABNORMAL, abnormal_test_label)
print("Test Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
# Predict labels for test data
predictions = model.predict(X_test_ABNORMAL)
predicted_labels = (predictions > 0.5).astype(int)
# Generate confusion matrix
conf_matrix = confusion_matrix(abnormal_test_label, predicted_labels)
# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False,
 xticklabels=['No Abnormality', 'Abnormality'],
 yticklabels=['No Abnormality', 'Abnormality'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


In [ ]:
# Combine ACL data from different planes
X_train_ACL = np.concatenate([axial_flat, coronal_flat, sagittal_flat])
X_test_ACL = np.concatenate([axial_test_flat, coronal_test_flat, sagittal_test_flat])


In [ ]:
# Define the CNN model
model = tf.keras.Sequential([
 tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Flatten(),
 tf.keras.layers.Dense(512, activation='relu'),
 tf.keras.layers.Dropout(0.5),
 tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Train the model
model.fit(X_train_ACL, acl_label, epochs=10, batch_size=32, validation_split=0.2)


In [ ]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test_ACL, acl_test_label)
print("Test Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
# Predict labels for test data
predictions = model.predict(X_test_ACL)
predicted_labels = (predictions > 0.5).astype(int)
# Generate confusion matrix
conf_matrix = confusion_matrix(acl_test_label, predicted_labels)
# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False,
 xticklabels=['No Abnormality', 'ACL'],
 yticklabels=['No Abnormality', 'ACL'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
#combine MENISCUS data from different planes
X_train_MENISCUS = [axial_flat, coronal_flat, sagittal_flat]
X_test_MENISCUS = [axial_test_flat, coronal_test_flat, sagittal_test_flat]


In [ ]:
# Define the CNN model
model = tf.keras.Sequential([
 tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Flatten(),
 tf.keras.layers.Dense(512, activation='relu'),
 tf.keras.layers.Dropout(0.5),
 tf.keras.layers.Dense(1, activation='sigmoid')
])


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Train the model
model.fit(X_train_MENISCUS, meniscus_label, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test_MENISCUS, meniscus_test_label)
print("Test Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
# Predict labels for test data
predictions = model.predict(X_test_MENISCUS)
predicted_labels = (predictions > 0.5).astype(int)
# Generate confusion matrix
conf_matrix = confusion_matrix(meniscus_test_label, predicted_labels)
# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False,
 xticklabels=['No Abnormality', 'Abnormality'],
 yticklabels=['No Abnormality', 'Abnormality'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel(